In [74]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import _stop_words
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
#from bertopic import BERTopic
import pandas as pd
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from langdetect import detect, DetectorFactory, LangDetectException
from gensim import corpora
from gensim.models.phrases import Phraser, Phrases
from gensim.models.ldamodel import LdaModel
import spacy
import ast

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Braider\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
rel_path = 'data/polarity_subjectivity_data_without_LemmStemm.csv'
csv_path = os.path.join(os.getcwd(), rel_path)
df_transcripts = pd.read_csv(csv_path)
df_transcripts

,Unnamed: 0,URL,transcript,Comedian Name,word_count,Unique ID,neg_polarity,neu_polarity,pos_polarity,compound,subjectivity
0,0,https://scrapsfromtheloft.com/comedy/pete-holm...,\n \n [audience cheering and applauding] \n He...,Pete Holmes,10068,0,0.056,0.688,0.256,1.0000,0.504545
1,1,https://scrapsfromtheloft.com/comedy/jeff-dunh...,"\n \n I’m funnier than he is, but they told me...",Jeff Dunham,4943,1,0.062,0.673,0.264,1.0000,0.535705
2,2,https://scrapsfromtheloft.com/comedy/taylor-to...,\n \n In her 2024 Netflix stand-up comedy spec...,Taylor Tomlinson,11197,2,0.077,0.717,0.206,1.0000,0.510567
3,3,https://scrapsfromtheloft.com/comedy/kevin-bri...,\n \n Kevin Bridges: The Overdue Catch-Up (20...,Kevin Bridges,12777,3,0.087,0.796,0.117,0.9999,0.504952
4,4,https://scrapsfromtheloft.com/comedy/jacquelin...,"\n \n In “Get on Your Knees,” Jacqueline Nova...",Jacqueline Novak,17783,4,0.077,0.733,0.190,1.0000,0.535544
...,...,...,...,...,...,...,...,...,...,...,...
468,468,https://scrapsfromtheloft.com/comedy/george-ca...,\n \n Jammin’ in New York is George Carlin’s 1...,George Carlin,8298,468,0.132,0.756,0.112,-0.9998,0.522932
469,469,https://scrapsfromtheloft.com/comedy/jim-jeffe...,\n \n Australian comedian Jim Jefferies breaks...,Jim Jefferies,2400,469,0.123,0.762,0.115,-0.9935,0.530056
470,470,https://scrapsfromtheloft.com/comedy/reggie-wa...,"\n \n Hello, I’m Thomas. I’m so glad to meet y...",Reggie Watts,4428,470,0.085,0.707,0.208,1.0000,0.562398
471,471,https://scrapsfromtheloft.com/comedy/george-ca...,\n \n Complaints and Grievances is a HBO stan...,George Carlin,8173,471,0.121,0.756,0.123,-0.9954,0.494151


## LDA

In [4]:
# Removing records that are not in english
DetectorFactory.seed = 0

def det_lang(transcript):
    try:
        return detect(transcript)
    except LangDetectException:
        return None
    
df_transcripts['language'] = df_transcripts['transcript'].apply(det_lang)

final_df = df_transcripts[df_transcripts['language'] == 'en']

final_df

,Unnamed: 0,URL,transcript,Comedian Name,word_count,Unique ID,neg_polarity,neu_polarity,pos_polarity,compound,subjectivity,language
0,0,https://scrapsfromtheloft.com/comedy/pete-holm...,\n \n [audience cheering and applauding] \n He...,Pete Holmes,10068,0,0.056,0.688,0.256,1.0000,0.504545,en
1,1,https://scrapsfromtheloft.com/comedy/jeff-dunh...,"\n \n I’m funnier than he is, but they told me...",Jeff Dunham,4943,1,0.062,0.673,0.264,1.0000,0.535705,en
2,2,https://scrapsfromtheloft.com/comedy/taylor-to...,\n \n In her 2024 Netflix stand-up comedy spec...,Taylor Tomlinson,11197,2,0.077,0.717,0.206,1.0000,0.510567,en
3,3,https://scrapsfromtheloft.com/comedy/kevin-bri...,\n \n Kevin Bridges: The Overdue Catch-Up (20...,Kevin Bridges,12777,3,0.087,0.796,0.117,0.9999,0.504952,en
4,4,https://scrapsfromtheloft.com/comedy/jacquelin...,"\n \n In “Get on Your Knees,” Jacqueline Nova...",Jacqueline Novak,17783,4,0.077,0.733,0.190,1.0000,0.535544,en
...,...,...,...,...,...,...,...,...,...,...,...,...
468,468,https://scrapsfromtheloft.com/comedy/george-ca...,\n \n Jammin’ in New York is George Carlin’s 1...,George Carlin,8298,468,0.132,0.756,0.112,-0.9998,0.522932,en
469,469,https://scrapsfromtheloft.com/comedy/jim-jeffe...,\n \n Australian comedian Jim Jefferies breaks...,Jim Jefferies,2400,469,0.123,0.762,0.115,-0.9935,0.530056,en
470,470,https://scrapsfromtheloft.com/comedy/reggie-wa...,"\n \n Hello, I’m Thomas. I’m so glad to meet y...",Reggie Watts,4428,470,0.085,0.707,0.208,1.0000,0.562398,en
471,471,https://scrapsfromtheloft.com/comedy/george-ca...,\n \n Complaints and Grievances is a HBO stan...,George Carlin,8173,471,0.121,0.756,0.123,-0.9954,0.494151,en


In [5]:
# Creating document term matrix 

cust_stop_words = ['goes', 'how', 'know', 'oh', 'got', 'said', 'like', 'just', 'right', 'we', 'got', 'going', 'go', 'get', 'one', 'yeah'
                   'gonna', 'okay']

eng_stop_words = stopwords.words('english')

total_stp_words = eng_stop_words + cust_stop_words
count_vectorizer = CountVectorizer(stop_words= total_stp_words)
count_data = count_vectorizer.fit_transform(final_df['transcript'])
df_tfd = pd.DataFrame(count_data.toarray(), columns = count_vectorizer.vocabulary_)
df_tfd.index = final_df['Comedian Name']
df_tfd


,audience,cheering,applauding,hello,applause,continue,thank,guys,much,minneapolis,...,condescended,patronized,goeth,suez,delusions,kufi,refuge,stipulations,amend,internment
Comedian Name,,,,,,,,,,,,,,,,,,,,,
Pete Holmes,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Jeff Dunham,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Taylor Tomlinson,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Kevin Bridges,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Jacqueline Novak,0,6,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
George Carlin,0,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Jim Jefferies,0,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Reggie Watts,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:

# Number of topics to extract
number_of_topics = 5

# Create and fit the LDA model
lda = LatentDirichletAllocation(n_components=number_of_topics, random_state = 55)
lda.fit(df_tfd)

# Function to display topics
def print_topics(model, vectorizer, num_top_words):
    words = vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(model.components_):
        print(f'Topic {topic_idx + 1}:')
        print(" ".join([words[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

# Number of words to display per topic
num_top_words = 10
print_topics(lda, count_vectorizer, num_top_words)

Topic 1:
crowd fucking dad mate wee bit guy man shite son
Topic 2:
cking lai rowan ck expletive plays americans bam drum hussein
Topic 3:
audience laughter laughing laughs people yeah think cheering time would
Topic 4:
people think yeah would gonna want time say really good
Topic 5:
shit man fuck gonna fucking people see want yeah cause


## NMF

In [7]:
# Create and fit the NMF model
number_of_topics = 5

nmf = NMF(n_components=number_of_topics, random_state=0)
nmf.fit(df_tfd)

# Function to display topics
def print_topics(model, vectorizer, num_top_words):
    words = vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(model.components_):
        print(f'Topic {topic_idx + 1}:')
        print(" ".join([words[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

# Number of words to display per topic
num_top_words = 10
print_topics(nmf, count_vectorizer, num_top_words)

Topic 1:
people think yeah would want gonna time really good say
Topic 2:
laughing audience applauding cheering crowd think little yeah time chuckles
Topic 3:
laughter applause yeah cheering people cheers fucking guy voice hey
Topic 4:
laughs audience crowd yeah chuckles people cheers cause want think
Topic 5:
shit fuck man fucking gonna ass white come see cause


## LSA

In [8]:
number_of_topics = 5
num_top_words = 10
# Create and fit the LSA model (also known as SVD in this context)
lsa = TruncatedSVD(n_components=number_of_topics, random_state=0)
lsa.fit(df_tfd)

def print_topics(model, vectorizer, num_top_words):
    words = vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(model.components_):
        print(f'Topic {topic_idx + 1}:')
        print(" ".join([words[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

print_topics(lsa, count_vectorizer, num_top_words)

Topic 1:
audience people gonna yeah laughing shit think man fucking laughter
Topic 2:
audience laughing laughs applauding cheering crowd chuckles continues clapping chuckling
Topic 3:
laughter applause audience laughs cheering cheers yeah voice chuckles mom
Topic 4:
laughs audience cheers want people applauds uh guys guy would
Topic 5:
shit fuck man fucking laughs motherfucker ass white black bitch


### Using Tokenized version

In [80]:
#Using the tokenized CSV 
rel_path = 'data/preprocess_final.csv'
csv_path = os.path.join(os.getcwd(), rel_path)
tokenized_df = pd.read_csv(csv_path)
tokenized_df

,Unnamed: 0,URL,transcript,Comedian Name,word_count,Unique ID,neg_polarity,neu_polarity,pos_polarity,compound,subjectivity,tokens_lst,select_tokens,all_tokens,diff
0,0,https://scrapsfromtheloft.com/comedy/pete-holm...,\n \n audience cheering and applauding \n hell...,Pete Holmes,10068,0,0.056,0.688,0.256,1.0000,0.504545,"['audience', 'cheering', 'applauding', 'hello'...","['audience', 'cheering', 'applauding', 'hello'...","[('audience', 'NN'), ('cheering', 'VBG'), ('ap...",553
1,1,https://scrapsfromtheloft.com/comedy/jeff-dunh...,\n \n im funnier than he is but they told me t...,Jeff Dunham,4943,1,0.062,0.673,0.264,1.0000,0.535705,"['im', 'funnier', 'told', 'introduce', 'heres'...","['im', 'funnier', 'told', 'introduce', 'heres'...","[('im', 'NN'), ('funnier', 'NN'), ('told', 'VB...",168
2,2,https://scrapsfromtheloft.com/comedy/taylor-to...,\n \n in her 2024 netflix standup comedy speci...,Taylor Tomlinson,11197,2,0.077,0.717,0.206,1.0000,0.510567,"['2024', 'netflix', 'standup', 'comedy', 'spec...","['netflix', 'standup', 'comedy', 'special', 't...","[('2024', 'CD'), ('netflix', 'JJ'), ('standup'...",565
3,3,https://scrapsfromtheloft.com/comedy/kevin-bri...,\n \n kevin bridges the overdue catchup 2023 ...,Kevin Bridges,12777,3,0.087,0.796,0.117,0.9999,0.504952,"['kevin', 'bridges', 'overdue', 'catchup', '20...","['kevin', 'bridges', 'overdue', 'catchup', 'co...","[('kevin', 'NN'), ('bridges', 'NNS'), ('overdu...",426
4,4,https://scrapsfromtheloft.com/comedy/jacquelin...,\n \n in get on your knees jacqueline novak ...,Jacqueline Novak,17783,4,0.077,0.733,0.190,1.0000,0.535544,"['get', 'knees', 'jacqueline', 'novak', 'trans...","['get', 'knees', 'jacqueline', 'novak', 'trans...","[('get', 'VB'), ('knees', 'NNS'), ('jacqueline...",611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,468,https://scrapsfromtheloft.com/comedy/george-ca...,\n \n jammin in new york is george carlins 14t...,George Carlin,8298,468,0.132,0.756,0.112,-0.9998,0.522932,"['jammin', 'new', 'york', 'george', 'carlins',...","['jammin', 'new', 'york', 'george', 'carlins',...","[('jammin', 'JJ'), ('new', 'JJ'), ('york', 'NN...",302
469,469,https://scrapsfromtheloft.com/comedy/jim-jeffe...,\n \n australian comedian jim jefferies breaks...,Jim Jefferies,2400,469,0.123,0.762,0.115,-0.9935,0.530056,"['australian', 'comedian', 'jim', 'jefferies',...","['australian', 'comedian', 'jim', 'jefferies',...","[('australian', 'JJ'), ('comedian', 'NN'), ('j...",98
470,470,https://scrapsfromtheloft.com/comedy/reggie-wa...,\n \n hello im thomas im so glad to meet you m...,Reggie Watts,4428,470,0.085,0.707,0.208,1.0000,0.562398,"['hello', 'im', 'thomas', 'im', 'glad', 'meet'...","['hello', 'im', 'thomas', 'im', 'glad', 'meet'...","[('hello', 'NN'), ('im', 'NN'), ('thomas', 'NN...",116
471,471,https://scrapsfromtheloft.com/comedy/george-ca...,\n \n complaints and grievances is a hbo stan...,George Carlin,8173,471,0.121,0.756,0.123,-0.9954,0.494151,"['complaints', 'grievances', 'hbo', 'standup',...","['complaints', 'grievances', 'hbo', 'standup',...","[('complaints', 'NNS'), ('grievances', 'NNS'),...",272


### bi-grams

In [81]:
tokenized_df['select_tokens'] = tokenized_df['select_tokens'].apply(ast.literal_eval)

bg = Phrases(tokenized_df['select_tokens'], min_count = 5, threshold = 10)
bg_mod = Phraser(bg)

tst_lst = [bg_mod[ts] for ts in tokenized_df['select_tokens']]

dic = corpora.Dictionary(tst_lst)
dic.filter_extremes(no_below = 5, no_above = 0.5)

corpus =  [dic.doc2bow(doc) for doc in tst_lst]

lda = LdaModel(corpus, num_topics = 5, id2word = dic, passes = 20)

topics = lda.print_topics(num_words = 10)

for x in topics:
    print(x)

(0, '0.030*"e" + 0.028*"di" + 0.019*"che" + 0.018*"un" + 0.017*"la" + 0.014*"non" + 0.013*"una" + 0.012*"il" + 0.009*"si" + 0.009*"le"')
(1, '0.008*"crowd_laughing" + 0.003*"cheers_applause" + 0.002*"doctor" + 0.002*"president" + 0.001*"cat" + 0.001*"pussy" + 0.001*"american" + 0.001*"goddamn" + 0.001*"balls" + 0.001*"ya"')
(2, '0.013*"audience_laughing" + 0.002*"sort" + 0.002*"chuckles" + 0.002*"laughs" + 0.001*"audience_cheering" + 0.001*"quite" + 0.001*"crowd_laughs" + 0.001*"fuckin" + 0.001*"ok" + 0.001*"youve_got"')
(3, '0.063*"laughter" + 0.022*"audience_laughs" + 0.005*"laughs" + 0.003*"cheering" + 0.003*"laughter_applause" + 0.003*"chuckles" + 0.003*"cheers_applause" + 0.002*"applause" + 0.002*"imitates" + 0.002*"cheering_applause"')
(4, '0.015*"aint" + 0.012*"n_g" + 0.011*"g" + 0.010*"motherfucker" + 0.008*"yall" + 0.005*"pussy" + 0.004*"goddamn" + 0.004*"white_people" + 0.004*"fuckin" + 0.003*"motherfuckers"')


### trigrams